In [2]:
import ee
import pandas as pd
from datetime import datetime

# Initialize the Earth Engine API
ee.Initialize()

# Define the region of interest (Aruba)
aruba = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Aruba'))

# Function to count Sentinel-2 images per month
def count_sentinel_images(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    
    # Load Sentinel-2 image collection and filter by date, region, and cloud coverage
    sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
        .filterDate(start_date, end_date) \
        .filterBounds(aruba) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  
    
    # Count the number of images
    count = sentinel2.size().getInfo()
    
    return count

# Create a list of years and months
current_year = datetime.now().year
years = range(2015, current_year + 1)
months = range(1, 13)

# Create a list to store the image counts
image_counts = []

# Loop over each year and month and get the image count
for year in years:
    for month in months:
        count = count_sentinel_images(year, month)
        image_counts.append({'year': year, 'month': month, 'image_count': count})

# Convert the results to a Pandas DataFrame
df = pd.DataFrame(image_counts)

# Display the DataFrame
print(df)

# Save the results to a CSV file
df.to_csv('output/sentinel2_aruba_monthly_image_count.csv', index=False)


     year  month  image_count
0    2015      1            0
1    2015      2            0
2    2015      3            0
3    2015      4            0
4    2015      5            0
..    ...    ...          ...
115  2024      8           18
116  2024      9            6
117  2024     10            0
118  2024     11            0
119  2024     12            0

[120 rows x 3 columns]


In [1]:
# Create cloud free composite images
import ee
from datetime import datetime

# Initialize the Earth Engine API
ee.Initialize()

# Define the region of interest (custom crudeBorders geometry)
crudeBorders = ee.Geometry.Polygon(
    [[[-70.07663362002641, 12.642409005331041],
      [-70.08350007510454, 12.528484055387077],
      [-69.95303742862016, 12.419873412682719],
      [-69.85416047549516, 12.389025068780875],
      [-69.84111421084673, 12.40109659488839],
      [-69.86240022158891, 12.486251996961656],
      [-70.05466096377641, 12.641069006200203]]]
)

def removeCloudsSentinel(image):
    MAX_CLOUD_PROBABILITY = 50

    S2CloudProb = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY") \
        .filter(ee.Filter.eq('system:index', image.get('system:index'))) \
        .first() \
        .select('probability')

    isNotCloud = S2CloudProb.lt(MAX_CLOUD_PROBABILITY)

    # Apply the cloud mask to the image
    return image.updateMask(isNotCloud)


# Function to create cloud-free composites for each period
def get_composite_images(start_date, end_date):
    # Load Sentinel-2 image collection
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(crudeBorders) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Filter cloud-free images

    cloudFree = sentinel2.map(removeCloudsSentinel)
    
    composite = cloudFree.median()
    
    # Add the date range as metadata to the composite
    composite = composite.set({
        'start_date': start_date.format('YYYY-MM-dd'),
        'end_date': end_date.format('YYYY-MM-dd')
    })

    composite.clip(crudeBorders)
    
    return composite

# Generate date ranges (every 3 months starting from April 2016)
# 8,9,10,11,12,1

start_year = 2016
start_month = 2
current_year = datetime.now().year
current_month = datetime.now().month

composite_images = []

for year in range(start_year, current_year + 1):
    for month in [2, 8]:  # Febr - Juli, Aug - Jan 
        if year == current_year and month > current_month:
            break  # Stop if the date exceeds the current month
        
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(6, 'month')
        
        # Get composite for the period and append to list
        composite_images.append(get_composite_images(start_date, end_date))

# Create an ImageCollection from the composite images
compositeImages = ee.ImageCollection(composite_images)

# Print the composite collection info
print(compositeImages.getInfo())


{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1,

In [37]:
# Exporting maps to Google Cloud

asset_folder = 'projects/aruba-tdf/assets/1-images/'

# Function to get the season index
def get_season(month):
    if month == 2:  
        return 'dry'
    elif month == 8:
        return 'wet'
    else:
        print('something goes wrong!')
    
# Function to export each image
def export_image_to_asset(image, index):
    start_date = ee.Date(image.get('start_date')).getInfo()
    year = start_date['value'] // 1000 // 31536000 + 1970  # Extract year from timestamp
    start_month = start_date['value'] // 1000 // 2628000 % 12 + 1  # Extract month from timestamp

    wet_dry = get_season(start_month)  # Get the season (0, 1, 2, 3)
    
    # Generate the name in the format {year}_{season_index}
    image_name = f'{year}_{wet_dry}'

    image = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])
    
    # Define the export task
    export_task = ee.batch.Export.image.toAsset(
        image=image,
        description=image_name,
        assetId=asset_folder + image_name,
        scale=10,  
        region=crudeBorders,
        crs='EPSG:4326',
        maxPixels=1e13  # Avoid "too many pixels" errors, adjust if needed
    )
    
    # Start the export task
    export_task.start()
    print(f'Started export for composite: {image_name}')

# Iterate over the images in the ImageCollection and export each one
composite_list = compositeImages.toList(compositeImages.size())
num_images = compositeImages.size().getInfo()

for i in range(num_images):
    image = ee.Image(composite_list.get(i))
    export_image_to_asset(image, i)


Started export for composite: 2018_dry
Started export for composite: 2024_dry
